In [2]:
#Importando as bibliotecas
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from requests_html import HTML
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import sqlite3
import datetime
import os

In [22]:
#Congiruando o driver
options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
options.add_argument('--no-sandbox')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [23]:
#Criando listas
Urls_amazon = []

Urls_amazon_more = []

Products_Links = []

Amazon_price = []
Amazon_price_2 = []

Amazon_seller = []
Amazon_seller_2 = []

Amazon_title = []
Amazon_title_2 = []

Amazon_installment_price_full = []
Amazon_installment_price_full_2 = []

Amazon_seller_more = []
Amazon_price_more = []
Amazon_title_more = []
internacional_list = []
more_offers_list = []

In [24]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)

C:\Users\pedro\AppData\Local\Temp\ipykernel_23592\1228600439.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)


In [25]:
#Função para criar os links de busca
def getting_n_creating_amazon(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "+")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.amazon.com.br/s?k=" + df['Urls']

    return df

In [26]:
Df_Products_raw = getting_n_creating_amazon("Motorola")
Df_Products_raw

,Brand,Product_Name,Product_Description,Urls,Urls_search
0,motorola,mbp481,babá eletrônica mbp481,motorola+mbp481,https://www.amazon.com.br/s?k=motorola+mbp481
1,motorola,mbp30a,babá eletrônica mbp30a,motorola+mbp30a,https://www.amazon.com.br/s?k=motorola+mbp30a
2,motorola,mbp36sc,babá eletrônica mbp36sc,motorola+mbp36sc,https://www.amazon.com.br/s?k=motorola+mbp36sc
3,motorola,mbp36xl,babá eletrônica mbp36xl,motorola+mbp36xl,https://www.amazon.com.br/s?k=motorola+mbp36xl
4,motorola,comfort85connect,babá eletrônica comfort85connect,motorola+comfort85connect,https://www.amazon.com.br/s?k=motorola+comfort...
5,motorola,lux64connect,babá eletrônica lux64connect,motorola+lux64connect,https://www.amazon.com.br/s?k=motorola+lux64co...
6,motorola,ease34,babá eletrônica ease34,motorola+ease34,https://www.amazon.com.br/s?k=motorola+ease34
7,motorola,ease44connect,babá eletrônica ease44connect,motorola+ease44connect,https://www.amazon.com.br/s?k=motorola+ease44c...
8,motorola,bliss54,babá eletrônica bliss54,motorola+bliss54,https://www.amazon.com.br/s?k=motorola+bliss54
9,motorola,comfort75,babá eletrônica comfort75,motorola+comfort75,https://www.amazon.com.br/s?k=motorola+comfort75


In [27]:
#Função pegando Links
def search_links(url):
    global Urls_amazon

    time.sleep(3)

    driver.get(url)
    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')
    html_obj = HTML(html=html_str)

    Links = [x for x in html_obj.links]
    products_links = [f'https://www.amazon.com.br{x}' for x in Links]

    Soup = BeautifulSoup(html_str,'html.parser')

    for link in products_links:
        Urls_amazon.append(link)

    Urls_amazon = [s for s in Urls_amazon if '/dp/' in s]
    Urls_amazon = [s for s in Urls_amazon if not '#customerReviews' in s]

In [28]:
for url in tqdm(Df_Products_raw['Urls_search']):
    search_links(url)

100%|██████████| 14/14 [01:05<00:00,  4.68s/it]


In [29]:
#Limpando as urls
def clean_link(urls, brand):
    #Criando a conexão com a tabela de exclusão 
    Database_exclusion = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_exclusao"

    #Criando a Query
    query = "SELECT * FROM " + table

    connection = sqlite3.connect(Database_exclusion)

    #Criando o dataset em brando
    df_itens = pd.read_sql_query(query, connection)

    clean_urls = pd.DataFrame()

    clean_urls['Urls_Completas'] = urls
    clean_urls['Urls_limpas'] = clean_urls['Urls_Completas'].str.partition("ref")[0]

    Urls_limpas = clean_urls['Urls_limpas'].tolist()

    for word in df_itens['Words']:
         Urls_limpas = [s for s in Urls_limpas if not word in s]

    clean_urls = pd.DataFrame()

    clean_urls['Urls_finais'] = Urls_limpas

    clean_urls['ASIN'] = clean_urls['Urls_finais'].str.partition("/dp/")[2].str.partition("/")[0]

    clean_urls.drop_duplicates(subset='ASIN',inplace=True)
    clean_urls.reset_index(inplace=True, drop=True)
    return clean_urls

In [30]:
len(Urls_amazon)

530

In [31]:
Urls_amazon

['https://www.amazon.com.br/Eletronica-Digital-C%C3%A2mera-Multikids-Baby/dp/B076LD2CMK/ref=sr_1_20?keywords=motorola+mbp481&qid=1652903439&sr=8-20&ufe=app_do%3Aamzn1.fos.25548f35-0de7-44b3-b28e-0f56f3f96147',
 'https://www.amazon.com.br/Carregador-Parede-20W-Sa%C3%ADda-MC-207/dp/B09C6NYKCG/ref=sr_1_46?keywords=motorola+mbp481&qid=1652903439&sr=8-46&ufe=app_do%3Aamzn1.fos.6a09f7ec-d911-4889-ad70-de8dd83c8a74',
 'https://www.amazon.com.br/Celular-Motorola-Moto-E7-Met%C3%A1lico/dp/B08WJP1QJ7/ref=sr_1_45?keywords=motorola+mbp481&qid=1652903439&sr=8-45',
 'https://www.amazon.com.br/Dados-Micro-USB-Lifeproof-MO-SKN6461AIBI-Preto/dp/B07B41GCHY/ref=sr_1_41?keywords=motorola+mbp481&qid=1652903439&sr=8-41',
 'https://www.amazon.com.br/Smartphone-Motorola-Moto-64GB-Ros%C3%AA/dp/B09LMRZV2W/ref=sr_1_39?keywords=motorola+mbp481&qid=1652903439&sr=8-39&ufe=app_do%3Aamzn1.fos.25548f35-0de7-44b3-b28e-0f56f3f96147',
 'https://www.amazon.com.br/Carregador-Parede-Motorola-100-240V-SC-47/dp/B08D3W1ZVT/ref=

In [32]:
New_urls = clean_link(Urls_amazon, 'Motorola')

In [33]:
New_urls.shape

(62, 2)

In [34]:
def search_atributes(url):
    #Tempo para não haver o bloqueio
    time.sleep(10)

    #Entrando dentro do site com o driver
    driver.get(url)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    #Criando o Soup
    soup = BeautifulSoup(html_str, 'html.parser')

    #Fazendo o try do nome do vendedor
    try:
        seller = soup.find(id='sellerProfileTriggerId').text
        seller_2 = soup.find("a", attrs={"id":'sellerProfileTriggerId'}).text
        Amazon_seller.append(seller)
        Amazon_seller_2.append(seller_2)
    except:
        Amazon_seller.append("Erro")
        Amazon_seller_2.append("Erro")

    #Fazendo o try do preço do produto a vista
    try:
        price = soup.find(class_='a-offscreen').text
        Amazon_price.append(price)

        Div_Price = soup.find('div', attrs={"data-feature-name":"corePrice"})
        price_2 = Div_Price.find(class_='a-offscreen').text
        Amazon_price_2.append(price_2)
    except:
        Amazon_price.append("Erro")
        Amazon_price_2.append("Erro")

    #Pegando o título do produto
    try:
        title = soup.find(id='productTitle').text
        Amazon_title.append(title)
    except:
        Amazon_title.append('Erro')

    #Pegando o internacional
    try:
        soup.find('img', attrs={'data-a-hires':'https://images-na.ssl-images-amazon.com/images/G/32/foreignseller/Foreign_Seller_Badge_v2._CB403622375_.png'})
        internacional_list.append("Internacional")
    except:
        internacional_list.append("Nacional")

    #Fazendo o try para pegar o preço da parcela
    try:
        installment = soup.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full.append(installment)
        
        Div_installment = soup.find('div', attrs={'id':'installmentCalculator_feature_div'})
        Installment_2 = Div_installment.find(class_='best-offer-name a-text-bold').text
        Amazon_installment_price_full_2.append(Installment_2)
    except:
        Amazon_installment_price_full.append("0")
        Amazon_installment_price_full_2.append("Erro")

    #Fazendo o try para ver se tem mais ofertas
    try:
        Main_Div_More_offers = soup.find('div', attrs={"id":"olpLinkWidget_feature_div"})
        Div_More_offers = Main_Div_More_offers.find('div', attrs={'class':'a-section olp-link-widget'})
        Div_More_offers_text = Div_More_offers.find('div', attrs={'class':'olp-text-box'}).text
        more_offers_list.append(Div_More_offers_text)
    except:
        more_offers_list.append("Comparar outras 0 ofertas")


In [35]:
for url in tqdm(New_urls['Urls_finais']):
    search_atributes(url)

100%|██████████| 62/62 [12:37<00:00, 12.21s/it]


In [36]:
print(len(Amazon_title))
print(len(Amazon_price_2))
print(len(Amazon_seller_2))
print(len(more_offers_list))
print(len(Amazon_installment_price_full_2))
print(len(internacional_list))

62
62
62
62
62
62


In [37]:
def dataset_amazon(url, sellers, preco, titulo, more_url, brand):
    # Criando o DataFrame
    Dataset_amazon = pd.DataFrame()

    #
    Hoje = pd.to_datetime('today', errors='ignore').date()

    # Colocando os dados
    Dataset_amazon['URL'] = url
    
    Dataset_amazon['DATE'] = Hoje

    Dataset_amazon['MARKETPLACE'] = "AMAZON"

    

    #Arrumando a coluna de sellers
    Dataset_amazon['SELLER'] = sellers
    Dataset_amazon['SELLER'] = Dataset_amazon['SELLER'].str.replace("Erro","Amazon", regex=False)

    #Arrumando o preço 
    Dataset_amazon['PRICE'] = preco
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(".","", regex=True)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace("R$","", regex=False)
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].str.replace(",",".", regex=True)
    

    #Arrumando os valores de installment 
    Dataset_amazon['INSTALLMENT FULL'] = Amazon_installment_price_full
    Dataset_amazon['PARCEL'] = Dataset_amazon['INSTALLMENT FULL'].str.extract('(\d+)')
    Dataset_amazon['PARCEL'] = Dataset_amazon['PARCEL'].astype("int")
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0]
    Dataset_amazon['Installment3'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['parcel_price_bruto'] = \
    Dataset_amazon['INSTALLMENT FULL'].str.partition("R$")[2].str.partition(" ")[2].str.partition(" ")[0].str.partition(",")[2]
    Dataset_amazon['Installment4'] = Dataset_amazon['parcel_price_bruto'].str.extract('(\d+)')
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['Installment3'] + "." + Dataset_amazon['Installment4']
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].astype("float")
    Dataset_amazon['INSTALLMENT'] = Dataset_amazon['INSTALLMENT'].fillna(0)
    Dataset_amazon['INSTALLMENT_PAYMENT'] = Dataset_amazon['PARCEL'] * Dataset_amazon['INSTALLMENT']


    Dataset_amazon['ID'] = Dataset_amazon['URL'].str.partition('/dp/')[2].str.partition('/')[0]
    Dataset_amazon['PRODUCT'] = titulo
    Dataset_amazon['INTERNACIONAL'] = internacional_list

    #Arrumando valores de mais sellers
    Dataset_amazon['MORE'] = more_url
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].str.partition("outras ")[2].str.partition(" ofertas")[0]
    
    Dataset_amazon['MORE'] = Dataset_amazon['MORE'].astype('int')
    

    Dataset_amazon = Dataset_amazon.drop(columns=["INSTALLMENT FULL", "Installment3", "Installment4", "parcel_price_bruto"])

    #Pegando os itens certos 
    Dataset_amazon = Dataset_amazon[Dataset_amazon["PRICE"] != "Erro"]
    Dataset_amazon['PRICE'] = Dataset_amazon['PRICE'].astype('float')


    #Colocando na ordem correta
    Dataset_amazon = Dataset_amazon[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL', 'INSTALLMENT', 'INSTALLMENT_PAYMENT', 'ID', 'PRODUCT','INTERNACIONAL', 'MORE']]

    if brand == "GoPro":
        Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 900]
    elif brand == 'Motorola':
        Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 70]
    elif brand == 'Wacom':
        Dataset_amazon = Dataset_amazon[Dataset_amazon['PRICE'] > 100]
    

    return Dataset_amazon

In [38]:
Df_final = dataset_amazon(New_urls['Urls_finais'], Amazon_seller_2, Amazon_price_2, Amazon_title, more_offers_list, 'Motorola')

In [39]:
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-05-18,https://www.amazon.com.br/Eletronica-Digital-C...,AMAZON,MMPLACE OFICIAL,426.90,10,42.69,426.90,B076LD2CMK,Babá Eletrônica Digital com Câmera BB1...,Internacional,17
2,2022-05-18,https://www.amazon.com.br/Eletr%C3%B4nica-Digi...,AMAZON,Planeta do Bebê,685.00,10,68.50,685.00,B07574YDHT,Babá Eletrônica Digital com Câmera Biv...,Internacional,12
4,2022-05-18,https://www.amazon.com.br/Motorola-Baba-Eletro...,AMAZON,Amazon,1299.00,1,1.93,1.93,B08FVXB74T,"Motorola Baba Eletronica Ease34, Motor...",Internacional,2
5,2022-05-18,https://www.amazon.com.br/C%C3%A2mera-3-2-Pole...,AMAZON,Canzile Shop,427.99,10,42.88,428.80,B081131PJQ,Câmera 3.2 Polegadas De Vídeo Sem Fio ...,Internacional,13
6,2022-05-18,https://www.amazon.com.br/Eletr%C3%B4nica-Digi...,AMAZON,MEIREMAGAZINE,348.97,10,34.96,349.60,B07HNPNC36,Babá Eletrônica Digital Com Video E Vi...,Internacional,9
7,2022-05-18,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,AMAZON,MOTOLATAM,1699.00,10,169.90,1699.00,B01N807E5P,Baba Eletrônica Motorola Mbp-855 Conne...,Internacional,2
8,2022-05-18,https://www.amazon.com.br/Motorola-MBP-36S-SC-...,AMAZON,Amazon,1599.90,10,159.99,1599.90,B00NSF730O,"Baba Eletrônica, Motorola, Branco",Internacional,2
9,2022-05-18,https://www.amazon.com.br/Bab%C3%A1-Eletr%C3%B...,AMAZON,MOTOLATAM,999.00,10,99.90,999.00,B078KHB3YW,Babá Eletrônica MBP668Connect com Tela...,Internacional,0
10,2022-05-18,https://www.amazon.com.br/Eletr%C3%B4nica-Moto...,AMAZON,Amazon,1592.91,10,159.30,1593.00,B01IRIVFRS,"Babá Eletrônica Motorola MBP36XL, Moto...",Internacional,0
12,2022-05-18,https://www.amazon.com.br/Dados-Motorola-Origi...,AMAZON,LOGMULTICOISAS,71.00,2,35.50,71.00,B0968P1CFC,Cabo De Dados Motorola Original Usb-C ...,Internacional,16


In [40]:
Df_final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Amazon_Motorola.xlsx", index=False)

In [21]:
Df_final.dtypes

DATE                    object
URL                     object
MARKETPLACE             object
SELLER                  object
PRICE                  float64
PARCEL                   int32
INSTALLMENT            float64
INSTALLMENT_PAYMENT    float64
ID                      object
PRODUCT                 object
INTERNACIONAL           object
MORE                     int32
dtype: object

In [22]:
Df_more_offers = Df_final[Df_final['MORE'] != 0]
Df_more_offers

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,INTERNACIONAL,MORE
0,2022-05-18,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,2999.99,10,300.08,3000.80,B01MQU5LW7,Mesa Digitalizadora Wacom Intuos Pro M...,Internacional,7
1,2022-05-18,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,5539.13,10,553.94,5539.40,B01MSXN9M5,Mesa Digitalizadora Intuos Pro Paper G...,Internacional,5
2,2022-05-18,https://www.amazon.com.br/Display-Interativo-W...,AMAZON,Amazon,3779.16,10,377.97,3779.70,B082LZXQ6B,"Display Interativo Wacom One, DTC133W0...",Internacional,9
3,2022-05-18,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,4849.91,10,485.00,4850.00,B01MTZ9CCD,Mesa Digitalizadora Wacom Intuos Pro L...,Internacional,5
4,2022-05-18,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,INPOWER,1329.00,10,132.90,1329.00,B079NWPZTF,Mesa Digitalizadora Wacom Média Blueto...,Internacional,9
5,2022-05-18,https://www.amazon.com.br/Mesa-Digitalizadora-...,AMAZON,Amazon,199.81,222,222.00,49284.00,B0785S55QQ,Wacom One CTL472 - Mesa Digitalizadora...,Internacional,46
6,2022-05-18,https://www.amazon.com.br/Display-Interativo-W...,AMAZON,MAGIC WORLD ELETRONICOS,3469.00,10,346.90,3469.00,B094489GDW,"Display Interativo Wacom One, DTC133W0...",Internacional,3
8,2022-05-18,https://www.amazon.com.br/Digitalizadora-Wacom...,AMAZON,Amazon,1743.87,1,1.90,1.90,B07PPQH867,"Mesa Digitalizadora Pro Wacom Intuos, ...",Internacional,10
9,2022-05-18,https://www.amazon.com.br/Caneta-Tablet-Wacom-...,AMAZON,Amazon,1099.90,1,1.00,1.00,B079NWPZTG,Mesa Digitalizadora Wacom Intuos Pen &...,Internacional,14
10,2022-05-18,https://www.amazon.com.br/Mesa-Digitalizadora-...,AMAZON,Megaprice Comercio,335.40,10,33.54,335.40,B079HL9YSF,Wacom CTL4100 - Mesa Digitalizadora In...,Internacional,33


In [27]:
Amazon_ID_More = []

In [28]:
def search_more_offers(ASIN):
    global Amazon_seller_more
    
    time.sleep(5)

    new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + ASIN + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(new_url)
    
    time.sleep(5)

    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-link-normal')[4:]:
        Amazon_seller_more.append(seller.text)
        

    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Política de devolução' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Apagar tudo' in s]
    Amazon_seller_more =  [s for s in Amazon_seller_more if len(s) > 1]

    for price in Soup.find_all(class_='a-offscreen')[2:]:
        Amazon_price_more.append(price.text)
        Amazon_ID_More.append(ASIN)

In [29]:
for url in tqdm(Df_more_offers['ID']):
    search_more_offers(url)

  3%|▎         | 1/32 [00:10<05:34, 10.80s/it]

https://www.amazon.com.br/gp/product/ajax/?asin=B01MQU5LW7&pageno=1&experienceId=aodAjaxMain


  6%|▋         | 2/32 [00:21<05:24, 10.81s/it]

https://www.amazon.com.br/gp/product/ajax/?asin=B01MSXN9M5&pageno=1&experienceId=aodAjaxMain


  9%|▉         | 3/32 [00:32<05:13, 10.81s/it]

...

https://www.amazon.com.br/gp/product/ajax/?asin=B084DVTSC9&pageno=1&experienceId=aodAjaxMain


 97%|█████████▋| 31/32 [05:36<00:10, 10.86s/it]

https://www.amazon.com.br/gp/product/ajax/?asin=B073WYP6VJ&pageno=1&experienceId=aodAjaxMain


100%|██████████| 32/32 [05:47<00:00, 10.86s/it]

https://www.amazon.com.br/gp/product/ajax/?asin=B01MS7ZXIJ&pageno=1&experienceId=aodAjaxMain


In [36]:
print(len(Amazon_seller_more))
print(len(Amazon_ID_More))
print(len(Amazon_price_more))

187
187
187


In [37]:
Dataframe_More = pd.DataFrame()
Dataframe_More['ID'] = Amazon_ID_More
Dataframe_More['Seller'] = Amazon_seller_more
Dataframe_More['Price'] = Amazon_price_more

Dataframe_More

,ID,Seller,Price
0,B01MQU5LW7,Tudo à Beça,"R$ 3.332,27"
1,B01MQU5LW7,Gigantec,"R$ 3.332,27"
2,B01MQU5LW7,INFOSHOP.,"R$ 3.364,41"
3,B01MQU5LW7,Thaun Shop,"R$ 3.409,77"
4,B01MQU5LW7,E-SPOT,"R$ 3.409,77"
...,...,...,...
182,B073WYP6VJ,Superfumaria,"R$ 252,11"
183,B073WYP6VJ,Sense Farma,"R$ 266,21"
184,B01MS7ZXIJ,Empório Charme,"R$ 269,99"
185,B01MS7ZXIJ,E-SPOT,"R$ 929,90"


In [38]:
Dataframe_More.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Amazon_More_Wacom.xlsx", index=False)

In [33]:
Amazon_seller_more =  [s for s in Amazon_seller_more if len(s) > 1]
print(len(Amazon_seller_more))

187


In [39]:
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Política de devolução' in s]
Amazon_seller_more =  [s for s in Amazon_seller_more if not 'Apagar tudo' in s]

print(len(Amazon_seller_more))

0


In [37]:
Amazon_seller_more

[' Tudo à Beça',
 'Política de devolução',
 ' Gigantec',
 'Política de devolução',
 ' INFOSHOP.',
 'Política de devolução',
 ' Thaun Shop',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 '',
 'Apagar tudo',
 ' O MUNDO PARA VOCÊ',
 'Política de devolução',
 ' ZAPATA STORE SP',
 'Política de devolução',
 ' OFICINA DOS BITS',
 'Política de devolução',
 ' Syma Solutions',
 'Política de devolução',
 ' Conect Magazine',
 'Política de devolução',
 ' IBYTE',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' Triplenet Pricing INC',
 'Política de devolução',
 ' MAGIC WORLD ELETRONICOS',
 'Política de devolução',
 ' REX INFORMATICA',
 'Política de devolução',
 ' Tudo à Beça',
 'Política de devolução',
 ' W-TECH IMPORTS',
 'Política de devolução',
 ' Kadri Tecnologia',
 'Política de devolução',
 ' E-SPOT',
 'Política de devolução',
 ' Tendinfo',
 'Política de devolução',
 '',
 'Apagar tudo',
 '

In [33]:
Amazon_ID_More = []

In [ ]:
Amazon_seller_more = []
Amazon_price_more = []


In [ ]:
Amazon_seller_more

In [ ]:
Amazon_price_more

In [ ]:

new_url = "https://www.amazon.com.br/gp/product/ajax/?asin=" + "B09D15SFMQ" + "&pageno=1&experienceId=aodAjaxMain"

driver.get(new_url)

time.sleep(5)

body_el = driver.find_element(By.CSS_SELECTOR, 'body')
html_str = body_el.get_attribute('innerHTML')

Soup = BeautifulSoup(html_str, 'html.parser')

for seller in Soup.find_all(class_='a-size-small a-link-normal')[4:]:
    print(seller.text)

for price in Soup.find_all(class_='a-offscreen')[2:]:
    print(price.text)


In [ ]:
for url in Df_more_offers['ID']:
    time.sleep(3)

    search = "https://www.amazon.com.br/gp/product/ajax/?asin=" + url + "&pageno=1&experienceId=aodAjaxMain"

    driver.get(search)
    body_el = driver.find_element(By.CSS_SELECTOR, 'body')
    html_str = body_el.get_attribute('innerHTML')

    print(search)

    Soup = BeautifulSoup(html_str, 'html.parser')

    for seller in Soup.find_all(class_='a-size-small a-link-normal'):
        print(seller.text)

In [ ]:
#Teste gráfico 
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
plt.boxplot(x=Df_final['PRICE'], vert=False)
plt.show()

In [ ]:
df_products = Df_final[Df_final['PRICE'] > 800]

In [ ]:
df_products